# Many to One Explainer - Demo of Conjunctive vs Disjunctive Explanations

This demo measures the differences in quality metrics and runtime of explanation generation between conjunctive and disjunctive explanations, on various datasets.

## Setup

In [2]:
import pandas as pd
import pd_explain
import time
import numpy as np
from sklearn.cluster import KMeans

usetex-False
(CVXPY) Jan 22 08:57:03 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 22 08:57:03 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


This comparison uses a very high, practically unrealistic separation error threshold of 0.8.\
This is set so that less explanations are filtered, and thus we don't get a situation where one form appears better than the other because most of its results were filtered out.\
You can of-course adjust this threshold to a more realistic value, such as 0.2 or 0.3, to get results that are more representative of a real-world scenario.

In [3]:
separation_error = 0.8

By default, we use sampling to speed up the explanation generation process.

In [4]:
pd_explain.get_use_sampling_value()

True

This can be changed either per function call or globally.\
If you would like to run this demo without sampling, uncomment and run the cell below prior to running the rest of the notebook.

In [5]:
# pd_explain.toggle_sampling(False)
# pd_explain.get_use_sampling_value()

In [20]:
def build_comparison_df(comparison_df, test_name, explanation_type, res_df, runtime):
    try:
        # Each group without an explanation has 1 row with "No explanation found" in the 'Explanation' column / index
        num_groups_not_found = res_df.loc[:, 'No explanation found', :].shape[0]
        # Each group that does have explanations can have multiple rows, and we want to count the number of groups, which is the length of the index on the first level
        num_groups_total = len(res_df.index.levels[0])
        percentage_not_found = (num_groups_not_found / num_groups_total) * 100
    # This should only happen in case explanations were found for all instances
    except KeyError:
        percentage_not_found = 0
    comparison_df.loc[(test_name, explanation_type), :] = [np.mean(res_df['Coverage']), np.mean(res_df['Separation Error']), percentage_not_found, runtime]

In [37]:
columns = ['Average Coverage', 'Average Separation Error', '% Explanation Not Found', 'Runtime']
index = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['Test', 'Explanation Type'])

## Adults dataset

In [7]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")
adults

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [8]:
adults_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [18]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels='label', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.2051966190338135


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...     0.75   
                0 <= capital-gain <= 4243.5                            0.99   
                0 <= capital-gain <= 5095.5                             1.0   
                1 <= education-num <= 10                               0.75   
>50K            0 <= capital-gain <= 99999                              1.0   
                marital-status == Married-civ-spouse                   0.84   
                8.5 <= education-num <= 16.0                           0.97   
                marital-status == Married-civ-spouse AND 8.5 <=...     0.81   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...             0.13  
                0 <= capital-gain <= 4243.5                                     0.2  
                0 <= capital-gain <= 5095.5                                    0.21  
                1 <= education-num <= 10                                       0.15  
>50K            0 <= capital-gain <= 99999                                     0.76  
                marital-status == Married-civ-spouse                           0.55  
                8.5 <= education-num <= 16.0                                   0.73  
                marital-status == Married-civ-spouse AND 8.5 <=...             0.52

In [19]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels='label', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.7175784111022949


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 4243.5                            0.99   
                1 <= education-num <= 10                               0.75   
                0 <= capital-gain <= 5095.5                             1.0   
>50K            marital-status == Married-civ-spouse                   0.84   
                marital-status == Married-civ-spouse OR 8.5 <= ...      1.0   
                0 <= capital-gain <= 99999                              1.0   
                8.5 <= education-num <= 16.0                           0.97   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 4243.5                                     0.2  
                1 <= education-num <= 10                                       0.15  
                0 <= capital-gain <= 5095.5                                    0.21  
>50K            marital-status == Married-civ-spouse                           0.55  
                marital-status == Married-civ-spouse OR 8.5 <= ...             0.74  
                0 <= capital-gain <= 99999                                     0.76  
                8.5 <= education-num <= 16.0                                   0.73

### Numeric only and categorical only tests

In [11]:
numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
labels = adults['label']

In [12]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.7494893074035645


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...     0.75   
                0 <= capital-gain <= 4243.5                            0.99   
                0 <= capital-gain <= 5095.5                             1.0   
                1 <= education-num <= 10                               0.75   
>50K            31.5 <= age <= 90 AND 8.5 <= education-num <= 16.0     0.87   
                0 <= capital-gain <= 99999                              1.0   
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90       0.72   
                8.5 <= education-num <= 16.0                           0.97   
                31.5 <= age <= 90                                       0.9   
                37 <= age <= 90                                        0.75   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-...             0.13  
                0 <= capital-gain <= 4243.5                                     0.2  
                0 <= capital-gain <= 5095.5                                    0.21  
                1 <= education-num <= 10                                       0.15  
>50K            31.5 <= age <= 90 AND 8.5 <= education-num <= 16.0             0.64  
                0 <= capital-gain <= 99999                                     0.76  
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90               0.62  
                8.5 <= education-num <= 16.0                                   0.73  
                31.5 <= age <= 90                                              0.67  
                37 <= age <= 90                                                0.65

In [13]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.990556001663208


Coverage  \
Group / Cluster Explanation                                                
<=50K           0 <= capital-gain <= 4243.5                         0.99   
                1 <= education-num <= 10                            0.75   
                0 <= capital-gain <= 5095.5                          1.0   
>50K            31.5 <= age <= 90                                    0.9   
                37 <= age <= 90                                     0.75   
                37 <= age <= 90 OR 8.5 <= education-num <= 16.0      1.0   
                0 <= capital-gain <= 99999                           1.0   
                8.5 <= education-num <= 16.0                        0.97   

                                                                Separation Error  
Group / Cluster Explanation                                                       
<=50K           0 <= capital-gain <= 4243.5                                  0.2  
                1 <= education-num <= 10                                    0.15  
                0 <= capital-gain <= 5095.5                                 0.21  
>50K            31.5 <= age <= 90                                           0.67  
                37 <= age <= 90                                             0.65  
                37 <= age <= 90 OR 8.5 <= education-num <= 16.0             0.74  
                0 <= capital-gain <= 99999                                  0.76  
                8.5 <= education-num <= 16.0                                0.73

In [14]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.15270662307739258


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Exec-managerial                          0.91   
                occupation != Prof-specialty                           0.91   
                occupation != Prof-specialty AND occupation != ...     0.82   
>50K            marital-status == Married-civ-spouse                   0.84   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Exec-managerial                                  0.21  
                occupation != Prof-specialty                                   0.21  
                occupation != Prof-specialty AND occupation != ...             0.17  
>50K            marital-status == Married-civ-spouse                           0.55

In [15]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.17460989952087402


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Exec-managerial                          0.91   
                occupation != Prof-specialty OR occupation != E...      1.0   
                occupation != Prof-specialty                           0.91   
>50K            marital-status == Married-civ-spouse                   0.84   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Exec-managerial                                  0.21  
                occupation != Prof-specialty OR occupation != E...             0.24  
                occupation != Prof-specialty                                   0.21  
>50K            marital-status == Married-civ-spouse                           0.55

### Groupby tests

In [16]:
single_attribute_groupby = adults.groupby('relationship').mean()
multi_attribute_groupby = adults.groupby(['sex', 'label']).mean()

In [21]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 4.567002058029175


Coverage  \
Group / Cluster Explanation                                                   
Not-in-family   marital-status != Married-civ-spouse AND 20.5 <...     0.96   
                marital-status != Married-civ-spouse AND 23.5 <...     0.88   
                marital-status != Married-civ-spouse                    1.0   
Husband         marital-status == Married-civ-spouse AND sex !=...      1.0   
                marital-status == Married-civ-spouse                    1.0   
                marital-status == Married-civ-spouse AND sex ==...      1.0   
Wife            marital-status == Married-civ-spouse AND sex ==...     0.99   
Own-child       marital-status != Married-civ-spouse AND 17 <= ...      0.9   
                17 <= age <= 31 AND marital-status == Never-mar...     0.78   
                marital-status != Married-civ-spouse AND 17 <= ...     0.76   
                marital-status != Married-civ-spouse AND 17.0 <...     0.82   
                17 <= age <= 37 AND marital-status == Never-mar...     0.84   
                marital-status != Married-civ-spouse AND 17.0 <...     0.88   
                17.0 <= age <= 53.5 AND marital-status == Never...     0.88   
                17.0 <= age <= 35.25 AND marital-status == Neve...     0.82   
                marital-status == Never-married                        0.89   
                17 <= age <= 28                                        0.76   
                17.0 <= age <= 41.33333333333333 AND marital-st...     0.86   
                marital-status != Married-civ-spouse AND 17.0 <...     0.94   
                marital-status != Married-civ-spouse AND 17.0 <...     0.98   
                marital-status != Married-civ-spouse                   0.98   
                marital-status != Married-civ-spouse AND 17 <= ...     0.82   
                17.0 <= age <= 31.6 AND marital-status == Never...     0.78   
                17 <= age <= 28 AND marital-status == Never-mar...     0.73   
Unmarried       marital-status != Married-civ-spouse AND 27.5 <...     0.88   
                marital-status != Married-civ-spouse AND 23.5 <...     0.95   
                marital-status != Married-civ-spouse AND marita...     0.76   
                marital-status != Married-civ-spouse AND 24.5 <...     0.94   
Other-relative  No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Not-in-family   marital-status != Married-civ-spouse AND 20.5 <...             0.47  
                marital-status != Married-civ-spouse AND 23.5 <...             0.44  
                marital-status != Married-civ-spouse                           0.52  
Husband         marital-status == Married-civ-spouse AND sex !=...             0.01  
                marital-status == Married-civ-spouse                           0.11  
                marital-status == Married-civ-spouse AND sex ==...             0.01  
Wife            marital-status == Married-civ-spouse AND sex ==...             0.05  
Own-child       marital-status != Married-civ-spouse AND 17 <= ...              0.6  
                17 <= age <= 31 AND marital-status == Never-mar...             0.49  
                marital-status != Married-civ-spouse AND 17 <= ...             0.47  
                marital-status != Married-civ-spouse AND 17.0 <...             0.53  
                17 <= age <= 37 AND marital-status == Never-mar...             0.54  
                marital-status != Married-civ-spouse AND 17.0 <...             0.58  
                17.0 <= age <= 53.5 AND marital-status == Never...             0.58  
                17.0 <= age <= 35.25 AND marital-status == Neve...             0.53  
                marital-status == Never-married                                0.59  
                17 <= age <= 28                                      

In [22]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.477092742919922


Coverage  \
Group / Cluster Explanation                                                   
Not-in-family   marital-status != Married-civ-spouse                    1.0   
Husband         marital-status == Married-civ-spouse                    1.0   
Own-child       17 <= age <= 28                                        0.76   
                marital-status == Never-married                        0.89   
                17 <= age <= 28 OR marital-status != Married-ci...     0.99   
                17 <= age <= 28 OR marital-status == Never-married     0.92   
                marital-status == Never-married OR 17 <= age <= 31     0.93   
                17.0 <= age <= 31.6 OR marital-status == Never-...     0.93   
                17.0 <= age <= 41.33333333333333 OR marital-sta...      1.0   
                marital-status != Married-civ-spouse                   0.98   
Other-relative  No explanation found                                    NaN   
Unmarried       No explanation found                                    NaN   
Wife            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Not-in-family   marital-status != Married-civ-spouse                           0.52  
Husband         marital-status == Married-civ-spouse                           0.11  
Own-child       17 <= age <= 28                                                0.56  
                marital-status == Never-married                                0.59  
                17 <= age <= 28 OR marital-status != Married-ci...             0.74  
                17 <= age <= 28 OR marital-status == Never-married             0.64  
                marital-status == Never-married OR 17 <= age <= 31             0.68  
                17.0 <= age <= 31.6 OR marital-status == Never-...             0.68  
                17.0 <= age <= 41.33333333333333 OR marital-sta...              0.8  
                marital-status != Married-civ-spouse                           0.72  
Other-relative  No explanation found                                            NaN  
Unmarried       No explanation found                                            NaN  
Wife            No explanation found                                            NaN

In [23]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.7219343185424805


Coverage  \
Group / Cluster     Explanation                                                   
('Male', '<=50K')   relationship != infrequent_sklearn AND 0 <= cap...     0.96   
                    0 <= capital-gain <= 5095.5                             1.0   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
('Female', '<=50K') relationship != Husband                                 1.0   
                    relationship != Husband AND 0 <= capital-gain <...     0.99   
                    relationship != Husband AND 0 <= capital-gain <...      1.0   
('Male', '>50K')    0 <= capital-gain <= 99999                              1.0   
                    8.5 <= education-num <= 16.0                           0.96   
                    relationship == Husband                                0.88   
                    relationship == Husband AND 8.5 <= education-nu...     0.85   
('Female', '>50K')  No explanation found                                    NaN   

                                                                       Separation Error  
Group / Cluster     Explanation                                                          
('Male', '<=50K')   relationship != infrequent_sklearn AND 0 <= cap...             0.49  
                    0 <= capital-gain <= 5095.5                                    0.51  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
('Female', '<=50K') relationship != Husband                                        0.51  
                    relationship != Husband AND 0 <= capital-gain <...             0.49  
                    relationship != Husband AND 0 <= capital-gain <...              0.5  
('Male', '>50K')    0 <= capital-gain <= 99999                                      0.8  
                    8.5 <= education-num <= 16.0                                   0.78  
                    relationship == Husband                                        0.56  
                    relationship == Husband AND 8.5 <= education-nu...             0.52  
('Female', '>50K')  No explanation found                                            NaN

In [24]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.8337545394897461


Coverage  \
Group / Cluster     Explanation                                                   
('Male', '<=50K')   0 <= capital-gain <= 5095.5                             1.0   
('Female', '<=50K') relationship != Husband                                 1.0   
('Male', '>50K')    relationship == Husband OR 8.5 <= education-num...      1.0   
                    relationship == Husband                                0.88   
                    0 <= capital-gain <= 99999                              1.0   
                    8.5 <= education-num <= 16.0                           0.96   
('Female', '>50K')  No explanation found                                    NaN   

                                                                       Separation Error  
Group / Cluster     Explanation                                                          
('Male', '<=50K')   0 <= capital-gain <= 5095.5                                    0.51  
('Female', '<=50K') relationship != Husband                                        0.51  
('Male', '>50K')    relationship == Husband OR 8.5 <= education-num...             0.78  
                    relationship == Husband                                        0.56  
                    0 <= capital-gain <= 99999                                      0.8  
                    8.5 <= education-num <= 16.0                                   0.78  
('Female', '>50K')  No explanation found                                            NaN

### Clustering test

#### 3 clusters

In [25]:
cluster_labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(adults))

C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  Fi

In [26]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.0711421966552734


Coverage  \
Group / Cluster Explanation                                                   
0               149278.5 <= fnlwgt <= 1490400 AND 8.5 <= educat...     0.87   
                149278.5 <= fnlwgt <= 1490400                           1.0   
1               291277.5 <= fnlwgt <= 1490400                           1.0   
2               13769 <= fnlwgt <= 149278.5                             1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               149278.5 <= fnlwgt <= 1490400 AND 8.5 <= educat...             0.21  
                149278.5 <= fnlwgt <= 1490400                                  0.22  
1               291277.5 <= fnlwgt <= 1490400                                   0.0  
2               13769 <= fnlwgt <= 149278.5                                     0.0

In [27]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 2.339325428009033


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,149278.5 <= fnlwgt <= 1490400,1.0,0.22
1,291277.5 <= fnlwgt <= 1490400,1.0,0.0
2,13769 <= fnlwgt <= 149278.5,1.0,0.0


#### 15 clusters

In [28]:
cluster_labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(adults))

In [29]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 7.512061357498169


Coverage  \
Group / Cluster Explanation                                                   
0               429223.0 <= fnlwgt <= 1490400 AND 17.0 <= age <...     0.71   
                429223.0 <= fnlwgt <= 1490400                           1.0   
1               131643.5 <= fnlwgt <= 165154.5                          1.0   
2               269888.0 <= fnlwgt <= 315581.0                          1.0   
3               13769 <= fnlwgt <= 60226.5                              1.0   
4               194360 <= fnlwgt <= 255559 AND 7.5 <= education...     0.91   
                194360 <= fnlwgt <= 255559                              1.0   
                194360 <= fnlwgt <= 255559 AND 4.5 <= education...     0.97   
5               899747.6000000001 <= fnlwgt <= 1195073.8                1.0   
6               97316.0 <= fnlwgt <= 131643.5                           1.0   
7               366566.0 <= fnlwgt <= 1490400                           1.0   
                366566.0 <= fnlwgt <= 1490400 AND 8.5 <= educat...     0.82   
8               157127 <= fnlwgt <= 194360                             0.95   
                157127 <= fnlwgt <= 194360 AND 24.5 <= age <= 90       0.83   
                140676 <= fnlwgt <= 208043                              1.0   
9               229539.0 <= fnlwgt <= 269888.0                          1.0   
10              604421.4 <= fnlwgt <= 899747.6000000001 AND 17 ...      1.0   
                604421.4 <= fnlwgt <= 899747.6000000001 AND 7.5...     0.75   
                604421.4 <= fnlwgt <= 899747.6000000001 AND 4.5...      1.0   
                604421.4 <= fnlwgt <= 899747.6000000001                 1.0   
                604421.4 <= fnlwgt <= 899747.6000000001 AND 8.5...     0.75   
11              60226.5 <= fnlwgt <= 97316.0                            1.0   
12              1195073.8 <= fnlwgt <= 1490400.0                        1.0   
                1121242.25 <= fnlwgt <= 1490400.0                       1.0   
                1244294.8333333333 <= fnlwgt <= 1490400.0               1.0   
                fnlwgt == 1490400                                       1.0   
                1261811.0 <= fnlwgt <= 1490400                          1.0   
13              315581.0 <= fnlwgt <= 1490400 AND 4.5 <= educat...     0.97   
                315581.0 <= fnlwgt <= 1490400                           1.0   
                315581.0 <= fnlwgt <= 1490400 AND 8.5 <= educat...     0.85   
14              505979.3333333333 <= fnlwgt <= 752084.5 AND 1 <...      0.8   
                512633.0 <= fnlwgt <= 1490400                           1.0   
                512633.0 <= fnlwgt <= 1490400 AND 1 <= educatio...      0.8   
                505979.3333333333 <= fnlwgt <= 752084.5 AND 31....      0.7   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               429223.0 <= fnlwgt <= 1490400 AND 17.0 <= age <...             0.29  
                429223.0 <= fnlwgt <= 1490400                                   0.3  
1               131643.5 <= fnlwgt <= 165154.5                                  0.0  
2               269888.0 <= fnlwgt <= 315581.0                                  0.0  
3               13769 <= fnlwgt <= 60226.5                                      0.0  
4               194360 <= fnlwgt <= 255559 AND 7.5 <= education...             0.31  
                194360 <= fnlwgt <= 255559                                     0.33  
                194360 <= fnlwgt <= 255559 AND 4.5 <= education...             0.32  
5               899747.6000000001 <= fnlwgt <= 1195073.8                        0.0  
6               97316.0 <= fnlwgt <= 131643.5                                   0.0  
7               366566.0 <= fnlwgt <= 1490400                                  0.41  
                366566.0 <= fnlwgt <= 1490400 AND 8.5 <= educat...             0.39  
8               15

In [30]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 4.69097375869751


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,429223.0 <= fnlwgt <= 1490400,1.0,0.3
1,131643.5 <= fnlwgt <= 165154.5,1.0,0.0
2,269888.0 <= fnlwgt <= 315581.0,1.0,0.0
3,13769 <= fnlwgt <= 60226.5,1.0,0.0
4,194360 <= fnlwgt <= 255559,1.0,0.33
5,899747.6000000001 <= fnlwgt <= 1195073.8,1.0,0.0
6,97316.0 <= fnlwgt <= 131643.5,1.0,0.0
7,366566.0 <= fnlwgt <= 1490400,1.0,0.41
8,140676 <= fnlwgt <= 208043,1.0,0.48


### Summary of results

In [31]:
conj_averages = adults_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = adults_comparison_df.loc[:, 'disjunctive', :].mean()
adults_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
adults_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [32]:
adults_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive               0.88875   
                         disjunctive              0.935714   
Numeric only             conjunctive                  0.87   
                         disjunctive                  0.92   
Categorical only         conjunctive                  0.87   
                         disjunctive                 0.915   
Single attribute groupby conjunctive                 0.885   
                         disjunctive                  0.94   
Multi attribute groupby  conjunctive                  0.96   
                         disjunctive              0.973333   
3 clusters               conjunctive                0.9675   
                         disjunctive                   1.0   
15 clusters              conjunctive              0.935588   
                         disjunctive              0.997368   
Average                  conjunctive              0.910977   
                         disjunctive              0.954488   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                       0.40625   
                         disjunctive                      0.477143   
Numeric only             conjunctive                         0.476   
                         disjunctive                       0.51375   
Categorical only         conjunctive                         0.285   
                         disjunctive                        0.3025   
Single attribute groupby conjunctive                      0.499643   
                         disjunctive                         0.604   
Multi attribute groupby  conjunctive                         0.565   
                         disjunctive                      0.656667   
3 clusters               conjunctive                        0.1075   
                         disjunctive                      0.073333   
15 clusters              conjunctive                      0.191471   
                         disjunctive                      0.146842   
Average                  conjunctive                      0.361552   
                         disjunctive                      0.396319   

                                          % Explanation Not Found   Runtime  
Test                     Explanation Type                                    
Full dataset             conjunctive                            0  1.205197  
                         disjunctive                            0  0.717578  
Numeric only             conjunctive                            0  0.749489  
                         disjunctive                            0  0.990556  
Categorical only         conjunctive                            0  0.152707  
                         disjunctive                            0   0.17461  
Single attribute groupby conjunctive                    16.666667  4.567002  
                         disjunctive                         50.0  3.477093  
Multi attribute groupby  conjunctive                         25.0  0.721934  
                         disjunctive                         25.0  0.833755  
3 clusters               conjunctive                            0  2.071142  
                         disjunctive                            0  2.339325  
15 clusters              conjunctive                            0  7.512061  
                         disjunctive                            0  4.690974  
Average                  conjunctive                     5.952381  2.425647  
                         disjunctive                    10.714286  1.889127

## Spotify dataset

In [33]:
spotify = pd.read_csv(r"..\Examples\Datasets\spotify_all.csv")
spotify

,acousticness,artists,danceability,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,speechiness,tempo,valence,year,decade,popularity_score,main_artist,duration_minutes
0,0.991000,['Mamie Smith'],0.598,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,...,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920,1920,10,Mamie Smith,2.805550
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,...,I Put A Spell On You,7,0.0534,86.889,0.9500,1920,1920,0,Screamin' Jay Hawkins,2.503333
2,0.993000,['Mamie Smith'],0.647,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,...,Golfing Papa,4,0.1740,97.600,0.6890,1920,1920,0,Mamie Smith,2.730450
3,0.000173,['Oscar Velazquez'],0.730,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,...,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920,1920,10,Oscar Velazquez,7.034783
4,0.295000,['Mixe'],0.704,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,...,Xuniverxe,2,0.0768,122.076,0.2990,1920,1920,0,Mixe,2.753733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,...,The One,0,0.0356,125.972,0.1860,2020,2020,0,DJ Combo,2.460250
174385,0.795000,['Alessia Cara'],0.429,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,...,A Little More,0,0.0360,94.710,0.2280,2021,2020,0,Alessia Cara,2.412000
174386,0.806000,['Roger Fly'],0.671,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,...,Together,0,0.0282,108.058,0.7140,2020,2020,0,Roger Fly,3.635783
174387,0.920000,['Taylor Swift'],0.462,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,...,champagne problems,69,0.0377,171.319,0.3200,2021,2020,60,Taylor Swift,4.066667


In [38]:
spotify_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [39]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.606048583984375


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= speechiness <= 0.194 AND 0 <= popularity...     0.75   
                0.0 <= speechiness <= 0.485                            0.97   
                0.0 <= speechiness <= 0.16166666666666665              0.91   
                0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= speechiness <= 0.3233333333333333 AND 0 ...     0.78   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= speechiness <= 0.194 AND 0 <= popularity...             0.01  
                0.0 <= speechiness <= 0.485                                    0.06  
                0.0 <= speechiness <= 0.16166666666666665                      0.03  
                0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= speechiness <= 0.3233333333333333 AND 0 ...             0.02  
1               No explanation found                                            NaN

In [40]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.8355743885040283


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= speechiness <= 0.2425 OR 0.0 <= populari...      1.0   
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity...      1.0   
                0.0 <= speechiness <= 0.16166666666666665 OR 0 ...     0.99   
                0.0 <= speechiness <= 0.485                            0.97   
                0.0 <= speechiness <= 0.194 OR 0.0 <= popularit...     0.99   
                0.0 <= speechiness <= 0.16166666666666665              0.91   
                0.0 <= speechiness <= 0.16166666666666665 OR 0....     0.99   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= speechiness <= 0.2425 OR 0.0 <= populari...             0.06  
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity...             0.06  
                0.0 <= speechiness <= 0.16166666666666665 OR 0 ...             0.05  
                0.0 <= speechiness <= 0.485                                    0.06  
                0.0 <= speechiness <= 0.194 OR 0.0 <= popularit...             0.05  
                0.0 <= speechiness <= 0.16166666666666665                      0.03  
                0.0 <= speechiness <= 0.16166666666666665 OR 0....             0.05  
1               No explanation found                                            NaN

### Numeric only and categorical only tests

In [42]:
numeric_only = spotify[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]
categorical_only = spotify[['name', 'main_artist', 'year']]
labels = spotify['explicit']

In [43]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.5166387557983398


Coverage  \
Group / Cluster Explanation                                          
0               0.0 <= speechiness <= 0.16166666666666665     0.91   
                0.0 <= speechiness <= 0.485                   0.97   
                0.0 <= speechiness <= 0.3233333333333333      0.95   
                0.0 <= speechiness <= 0.2425                  0.94   
1               No explanation found                           NaN   

                                                          Separation Error  
Group / Cluster Explanation                                                 
0               0.0 <= speechiness <= 0.16166666666666665             0.03  
                0.0 <= speechiness <= 0.485                           0.06  
                0.0 <= speechiness <= 0.3233333333333333              0.05  
                0.0 <= speechiness <= 0.2425                          0.04  
1               No explanation found                                   NaN

In [44]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.8359169960021973


Coverage  \
Group / Cluster Explanation                                                   
0               0.0 <= speechiness <= 0.3233333333333333               0.95   
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0...      1.0   
                0.0 <= speechiness <= 0.2425                           0.94   
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= spee...      1.0   
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= sp...      1.0   
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...      1.0   
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...      1.0   
                0.0 <= speechiness <= 0.485                            0.97   
                0.0 <= speechiness <= 0.16166666666666665              0.91   
1               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               0.0 <= speechiness <= 0.3233333333333333                       0.05  
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0...             0.07  
                0.0 <= speechiness <= 0.2425                                   0.04  
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= spee...             0.07  
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= sp...             0.07  
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...             0.07  
                0.0 <= instrumentalness <= 0.4995 OR 0.0 <= spe...             0.07  
                0.0 <= speechiness <= 0.485                                    0.06  
                0.0 <= speechiness <= 0.16166666666666665                      0.03  
1               No explanation found                                            NaN

In [45]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.6001811027526855


Coverage  \
Group / Cluster Explanation                                           
0               name == infrequent_sklearn                      1.0   
                main_artist == artist_infrequent_sklearn        1.0   
1               No explanation found                            NaN   

                                                           Separation Error  
Group / Cluster Explanation                                                  
0               name == infrequent_sklearn                             0.07  
                main_artist == artist_infrequent_sklearn               0.07  
1               No explanation found                                    NaN

In [46]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.738072395324707


Coverage  \
Group / Cluster Explanation                                         
0               name == infrequent_sklearn                    1.0   
                main_artist == artist_infrequent_sklearn      1.0   
1               No explanation found                          NaN   

                                                         Separation Error  
Group / Cluster Explanation                                                
0               name == infrequent_sklearn                           0.07  
                main_artist == artist_infrequent_sklearn             0.07  
1               No explanation found                                  NaN

### Groupby tests

In [48]:
single_attribute_groupby = spotify.groupby('mode').mean()

In [49]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.2848331928253174


Coverage  \
Group / Cluster Explanation                                                   
1               0.0 <= speechiness <= 0.3233333333333333 AND 0....     0.73   
                0.0 <= speechiness <= 0.485                            0.95   
                0.07324999943375587 <= acousticness <= 0.996           0.78   
0               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
1               0.0 <= speechiness <= 0.3233333333333333 AND 0....             0.27  
                0.0 <= speechiness <= 0.485                                     0.3  
                0.07324999943375587 <= acousticness <= 0.996                   0.28  
0               No explanation found                                            NaN

In [50]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 2.7960195541381836


Coverage  \
Group / Cluster Explanation                                                   
1               0.07324999943375587 <= acousticness <= 0.996 OR...      1.0   
                0.0 <= speechiness <= 0.485                            0.95   
                0.07324999943375587 <= acousticness <= 0.996           0.78   
0               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
1               0.07324999943375587 <= acousticness <= 0.996 OR...              0.3  
                0.0 <= speechiness <= 0.485                                     0.3  
                0.07324999943375587 <= acousticness <= 0.996                   0.28  
0               No explanation found                                            NaN

### Clustering test

#### 3 clusters

In [54]:
# The spotify dataset is too big to cluster all columns with get_dummies (I tried, got a MemoryError, saying it is unable to allocate 22.3 GiB)
cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

In [55]:
cluster_labels = KMeans(n_clusters=3).fit_predict(spotify[cols])

In [56]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.4809114933013916


Coverage  \
Group / Cluster Explanation                                                  
0               147.00550079345703 <= tempo <= 219.446                 1.0   
1               0.0 <= tempo <= 106.19950103759766                    0.99   
                0.0 <= tempo <= 106.32799911499023                    0.99   
                0.0 <= tempo <= 109.723                                1.0   
2               106.73249816894531 <= tempo <= 147.00550079345703      1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               147.00550079345703 <= tempo <= 219.446                         0.0  
1               0.0 <= tempo <= 106.19950103759766                             0.0  
                0.0 <= tempo <= 106.32799911499023                             0.0  
                0.0 <= tempo <= 109.723                                       0.08  
2               106.73249816894531 <= tempo <= 147.00550079345703              0.0

In [57]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.2725210189819336


Coverage  \
Group / Cluster Explanation                                                  
0               147.00550079345703 <= tempo <= 219.446                 1.0   
1               0.0 <= tempo <= 106.32799911499023                    0.99   
                0.0 <= tempo <= 109.723                                1.0   
                0.0 <= tempo <= 106.19950103759766                    0.99   
2               106.73249816894531 <= tempo <= 147.00550079345703      1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               147.00550079345703 <= tempo <= 219.446                         0.0  
1               0.0 <= tempo <= 106.32799911499023                             0.0  
                0.0 <= tempo <= 109.723                                       0.08  
                0.0 <= tempo <= 106.19950103759766                             0.0  
2               106.73249816894531 <= tempo <= 147.00550079345703              0.0

#### 15 clusters

In [59]:
cluster_labels = KMeans(n_clusters=15).fit_predict(spotify[cols])

In [60]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 9.19522213935852


Coverage  \
Group / Cluster Explanation                                                   
0               100.64099884033203 <= tempo <= 113.462001800537...     0.86   
                100.64099884033203 <= tempo <= 113.46200180053711       1.0   
                100.64099884033203 <= tempo <= 113.462001800537...     0.99   
                100.64099884033203 <= tempo <= 113.462001800537...     0.96   
                100.58349990844727 <= tempo <= 113.497501373291...     0.96   
...                                                                     ...   
14              90.001 <= tempo <= 107.148 AND -60.0 <= loudnes...     0.93   
                92.04199981689453 <= tempo <= 107.2234992980957...     0.78   
                92.04199981689453 <= tempo <= 107.2234992980957...     0.92   
                90.001 <= tempo <= 107.148 AND -60.0 <= loudnes...     0.79   
5               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               100.64099884033203 <= tempo <= 113.462001800537...             0.02  
                100.64099884033203 <= tempo <= 113.46200180053711              0.28  
                100.64099884033203 <= tempo <= 113.462001800537...             0.11  
                100.64099884033203 <= tempo <= 113.462001800537...             0.07  
                100.58349990844727 <= tempo <= 113.497501373291...             0.07  
...                                                                             ...  
14              90.001 <= tempo <= 107.148 AND -60.0 <= loudnes...             0.25  
                92.04199981689453 <= tempo <= 107.2234992980957...             0.07  
                92.04199981689453 <= tempo <= 107.2234992980957...             0.19  
                90.001 <= tempo <= 107.148 AND -60.0 <= loudnes...             0.15  
5               No explanation found                                            NaN  

[61 rows x 2 columns]

In [61]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 6.80797004699707


Coverage  \
Group / Cluster Explanation                                                  
0               100.64099884033203 <= tempo <= 113.46200180053711      1.0   
                100.58349990844727 <= tempo <= 113.49750137329102      1.0   
2               165.96400451660156 <= tempo <= 219.446                 1.0   
7               0.0 <= tempo <= 36.815500259399414                     1.0   
                0.0 <= tempo <= 36.574333333333335                     1.0   
8               186.69149780273438 <= tempo <= 219.446                 1.0   
10              36.574333333333335 <= tempo <= 73.14866666666667      0.93   
                36.815500259399414 <= tempo <= 73.3915023803711       0.95   
11              86.73649978637695 <= tempo <= 100.49849700927734       1.0   
12              113.49750137329102 <= tempo <= 124.86849975585938     0.99   
13              137.57550048828125 <= tempo <= 150.56100463867188     0.95   
1               No explanation found                                   NaN   
3               No explanation found                                   NaN   
4               No explanation found                                   NaN   
5               No explanation found                                   NaN   
6               No explanation found                                   NaN   
9               No explanation found                                   NaN   
14              No explanation found                                   NaN   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               100.64099884033203 <= tempo <= 113.46200180053711             0.28  
                100.58349990844727 <= tempo <= 113.49750137329102             0.28  
2               165.96400451660156 <= tempo <= 219.446                        0.21  
7               0.0 <= tempo <= 36.815500259399414                             0.0  
                0.0 <= tempo <= 36.574333333333335                             0.0  
8               186.69149780273438 <= tempo <= 219.446                         0.0  
10              36.574333333333335 <= tempo <= 73.14866666666667              0.09  
                36.815500259399414 <= tempo <= 73.3915023803711                0.1  
11              86.73649978637695 <= tempo <= 100.49849700927734              0.29  
12              113.49750137329102 <= tempo <= 124.86849975585938             0.22  
13              137.57550048828125 <= tempo <= 150.56100463867188             0.08  
1               No explanation found                                           NaN  
3               No explanation found                                           NaN  
4               No explanation found                                           NaN  
5               No explanation found                                           NaN  
6               No explanation found                                           NaN  
9               No explanation found                                           NaN  
14              No explanation found                                           NaN

### Summary of results

In [62]:
conj_averages = spotify_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = spotify_comparison_df.loc[:, 'disjunctive', :].mean()
spotify_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
spotify_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [63]:
spotify_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive              0.883333   
                         disjunctive              0.971111   
Numeric only             conjunctive                0.9425   
                         disjunctive              0.974444   
Categorical only         conjunctive                   1.0   
                         disjunctive                   1.0   
Single attribute groupby conjunctive                  0.82   
                         disjunctive                  0.91   
3 clusters               conjunctive                 0.996   
                         disjunctive                 0.996   
15 clusters              conjunctive              0.923833   
                         disjunctive              0.983636   
Average                  conjunctive              0.927611   
                         disjunctive              0.972532   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                         0.035   
                         disjunctive                          0.05   
Numeric only             conjunctive                         0.045   
                         disjunctive                      0.058889   
Categorical only         conjunctive                          0.07   
                         disjunctive                          0.07   
Single attribute groupby conjunctive                      0.283333   
                         disjunctive                      0.293333   
3 clusters               conjunctive                         0.016   
                         disjunctive                         0.016   
15 clusters              conjunctive                      0.113167   
                         disjunctive                      0.140909   
Average                  conjunctive                       0.09375   
                         disjunctive                      0.104855   

                                          % Explanation Not Found   Runtime  
Test                     Explanation Type                                    
Full dataset             conjunctive                         50.0  1.606049  
                         disjunctive                         50.0  1.835574  
Numeric only             conjunctive                         50.0  1.516639  
                         disjunctive                         50.0  1.835917  
Categorical only         conjunctive                         50.0  0.600181  
                         disjunctive                         50.0  0.738072  
Single attribute groupby conjunctive                         50.0  2.284833  
                         disjunctive                         50.0   2.79602  
3 clusters               conjunctive                            0  2.480911  
                         disjunctive                            0  3.272521  
15 clusters              conjunctive                     6.666667  9.195222  
                         disjunctive                    46.666667   6.80797  
Average                  conjunctive                    34.444444  2.947306  
                         disjunctive                    41.111111  2.881012